<a href="https://colab.research.google.com/github/sadrakhamoshi/CI-Course/blob/main/CI992_HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Your info

Full name: Sadra Khamoshi

Student ID:97521261

Notice: **You can add new cells for each part.**

# Q1

In [ ]:
#write your code here

import random

neighborhood_distance = {
    (1, 2): 12,
    (1, 3): 10,
    (1, 7): 12,
    (2, 4): 12,
    (2, 3): 8,
    (3, 4): 11,
    (3, 5): 3,
    (3, 7): 9,
    (4, 5): 11,
    (4, 6): 10,
    (5, 6): 6,
    (5, 7): 7,
    (6, 7): 9
}


def get_distance(source, dist):
    if neighborhood_distance.get((source, dist)):
        return neighborhood_distance.get((source, dist))

    if neighborhood_distance.get((dist, source)):
        return neighborhood_distance.get((dist, source))

    return float('inf')


def fitness_function(cycle):
    distance = 0
    for i in range(len(cycle)):
        from_city = cycle[i]
        if i + 1 >= len(cycle):
            to_city = cycle[0]
        else:
            to_city = cycle[i + 1]
        distance += get_distance(source=from_city, dist=to_city)
    fitness = 1.0 / float(distance)
    return fitness


class GA:
    def __init__(self, cities, population_size, num_generations):
        self.cities = cities
        self.population_size = population_size
        self.num_generations = num_generations

    def initial_population(self, population_size, cities):
        population = []
        
        # to don't give us undifined path
        population.append([1,3,2,4,5,6,7])

        for _ in range(population_size-1):
            individual = random.sample(cities, len(cities))
            population.append(individual)
        return population

    def rankedPopulation(self, population):
        # chromosome and index tuple
        pop_fitness = [(i, fitness_function(population[i]))
                       for i in range(len(population))]

        # sort population by fitness
        sorted_by_fitness = sorted(pop_fitness,
                                   key=lambda tup: tup[1],
                                   reverse=True)

        return sorted_by_fitness

    def selection(self, population, elite_size):
        sorted_pop = self.rankedPopulation(population)
        selection_idx = []

        fitness_sum = 0
        for k, v in sorted_pop:
            fitness_sum += v

        for i in range(elite_size):
            selection_idx.append(sorted_pop[i][0])

        for i in range(0, len(sorted_pop) - elite_size):
            pick = 100 * random.random()
            for i in range(0, len(sorted_pop)):
                percent = (sorted_pop[i][1] / float(fitness_sum)) * 100 if fitness_sum!=0 else -1
                if pick <= percent:
                    selection_idx.append(sorted_pop[i][0])
                    break

        selection_result = [population[i] for i in selection_idx]
        return selection_result

    def crossover(self, parent1, parent2):
        idexs = random.sample(range(len(parent1)), 2)
        startGene = min(idexs)
        endGene = max(idexs)
        before = []
        middle = parent1[startGene:endGene]
        after = []

        for item in parent2:
            if item not in middle:
                if len(before) < startGene:
                    before.append(item)
                else:
                    after.append(item)

        offspring = before + middle + after
        return offspring

    def crossoverPopulation(self, population, elite_size):
        childrens = []
        for i in range(elite_size):
            childrens.append(population[i])

        for i in range(len(population) - elite_size):
            parents = random.sample(population, 2)
            offspring = self.crossover(parents[0], parents[1])
            childrens.append(offspring)

        return childrens

    def mutate(self, population, elite_size, mutation_rate):
        mutated_pop = population[:elite_size]

        for chromosome in population[elite_size:]:
            for swapped in range(len(chromosome)):
                if random.random() < mutation_rate:
                    swap_with = int(random.random() * len(chromosome))

                    item1 = chromosome[swapped]
                    item2 = chromosome[swap_with]
                    chromosome[swapped] = item2
                    chromosome[swap_with] = item1

            mutated_pop.append(chromosome)

        return mutated_pop

    def Genetic_algorithm(self, elite_size=15, mutation_rate=0.01):
        population = self.initial_population(self.population_size,
                                              self.cities)
        fitness = self.rankedPopulation(population)[0][1]

        fitness = 1/fitness if fitness != 0 else float('inf')
        print(f'The Initiali Distance is : {fitness}')

        for i in range(self.num_generations):
            # selection
            selected = self.selection(population, elite_size)

            # crossover
            childrens = self.crossoverPopulation(selected, elite_size)

            # mutation
            population = self.mutate(childrens, elite_size, mutation_rate)

            sorted_pop = self.rankedPopulation(population)
            fitness = sorted_pop[0][1]

        print(f"\nThe Final distance is : {1/fitness if fitness != 0 else float('inf')}")


# Generate cities and distances between them
cities = list(range(1, 8))

tsp = GA(cities=cities, population_size=60, num_generations=500)

tsp.Genetic_algorithm(elite_size=15, mutation_rate=0.05)

The Initiali Distance is : 65.0

The Final distance is : 63.0


# Q2

In [ ]:

#write your code here

import random
import numpy as np

population_size = 100
generation_size = 300
mutation_rate = 0.04
crossover_rate = 0.7


def equation(x):
    return 9 * (x)**5 - 194.7 * (x)**4 + 1680.1 * (x)**3 - 7227.94 * (
        x)**2 + 15501.2 * (x) - 13257.2


def binaryStr2Float(chromosome):
    decimal = int(chromosome[1:8], 2)
    floating = int(chromosome[8:], 2)
    num = decimal + (floating / (10**len(str(floating))))
    if chromosome[1] == "1":
        num = -num
    return num


def fitness_calculation(chromosome):
    decimal = binaryStr2Float(chromosome)
    f = equation(decimal)
    return abs(f)


def mutation(child):
    random_idx = random.randint(0, len(child) - 1)
    if child[random_idx] == '0':
        child[random_idx] = '1'
    else:
        child[random_idx] = '0'
    return child


def crossover(par1, par2):

    parent1 = list(str(par1[0]))
    parent2 = list(str(par2[0]))

    rand_indexes = random.sample(range(0, len(parent1)), 2)

    startGene = min(rand_indexes)
    endGene = max(rand_indexes)

    parent2[startGene:endGene] = parent1[startGene:endGene]
    child = parent2

    # mutation
    if random.random() < mutation_rate:
        child = mutation(child)
    child = "".join(child)
    return (child, fitness_calculation(child))


def initialPopulation():
    population = []
    for _ in range(population_size):
        random_list = np.random.randint(2, size=14)
        chromosome = ''.join(map(str, random_list))
        population.append((chromosome, fitness_calculation(chromosome)))
    return population


def selection(population, fitest_chromosome_size, crossover_size):
    new_generation = []
    # pick top fitest parent for next generation
    new_generation.extend(population[:fitest_chromosome_size])

    # do some crossover with top half of fitest parent
    half_size = int(0.5 * population_size)

    for _ in range(crossover_size):
        parent1 = random.choice(population[:half_size])
        parent2 = random.choice(population[:half_size])
        child = crossover(parent1, parent2)
        new_generation.append(child)
    return new_generation


def GenerateEvolution(population):
    crossover_size = int(crossover_rate * population_size)

    fitest_chromosome_size = population_size - crossover_size

    # sort population by fitness in increasing order
    population = sorted(population, key=lambda x: x[1])

    # selection :
    new_generation = selection(population, fitest_chromosome_size,
                               crossover_size)

    return new_generation


def main():
    population = initialPopulation()
    init_pop = sorted(population, key=lambda x: x[1])

    decimal_number = binaryStr2Float(init_pop[0][0])
    print(
        f'Initiali awnser for x : {decimal_number}  with value of f(x) = {str(init_pop[0][1])} \n'
    )

    for gen in range(generation_size):
        population = GenerateEvolution(population)

    decimal_number = binaryStr2Float(population[0][0])
    print(
        f'The best solution found in x : {decimal_number}  with value of f(x) = {str(population[0][1])}'
    )


if __name__ == '__main__':
    main()

Initiali awnser for x : 4.45  with value of f(x) = 0.1370540624957357 

The best solution found in x : 4.9  with value of f(x) = 0.02244000001155655


# <font color='red'>Submission</font>

1. Sign up in [Gradescope](https://www.gradescope.com) with proper name and student ID and use the following code to join the class: <font color='red'>**D5372R**</font>
2. Fill in your full name (seperated by single spaces) and student ID in the beginning of this notebook.
3. After you're done with this notebook, you should do the following:
  - Clear all outputs of the notebook.
  ![clear all outputs](https://i.ibb.co/y6FrttB/Screen-Shot-2021-03-21-at-01-51-42.png)
  - Run all of the cells (if you skipped a question just leave the cell unchanged), and make sure all of your outputs are correct.
  ![run all](https://i.ibb.co/cgRcBZ0/Screen-Shot-2021-03-21-at-01-54-58.png)
  - Save your notebook.
  
  - If you're using Colab, download your notebook.
  ![download ipynb](https://i.ibb.co/2KxYM6K/Screen-Shot-2021-03-21-at-02-03-50.png)
  
  - Put the notebook file you just downloaded and `convert.py` in the same folder run the following command:
  ```bash
  python convert.py
  ```
  This will export your code for each question into a `.py` file.
   

  according to the question number.
  - There are 2 assignments in Gradescope: 

    You should upload your **codes** and your **notebook** in `HW4` section and your final report for all of the questions as a **single pdf** file in `HW4 - Report`. Autograder will automatically check for:
    - `CI992_HW4.ipynb`
    - `Q1 Q1.py`
    - `Q2 Q2.py`
    - Your name and ID in the beginning of `.ipynb` file.

    It is important that you <font color='red'>**don't**</font> change the names of these files before submission.

4. If you pass the autograder, you're good to go.